In [3]:
import serial
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy import interpolate
# %matplotlib notebook

%matplotlib tk

def read_arduino(ser,inputBufferSize):
#    data = ser.readline(inputBufferSize)
    data = ser.read(inputBufferSize)
    out =[(int(data[i])) for i in range(0,len(data))]
    return out

def process_data(data):
    data_in = np.array(data)
    result = []
    i = 1
    while i < len(data_in)-1:
        if data_in[i] > 127:
            # Found beginning of frame
            # Extract one sample from 2 bytes
            intout = (np.bitwise_and(data_in[i],127))*128
            i = i + 1
            intout = intout + data_in[i]
            result = np.append(result,intout)
        i=i+1
    return result

In [4]:
# List available ports
import serial.tools.list_ports
ports = serial.tools.list_ports.comports()

for port, desc, hwid in sorted(ports):
        print("{}: {} [{}]".format(port, desc, hwid))

COM3: USB Serial Port (COM3) [USB VID:PID=0403:6015 SER=DM89OND5A]


In [5]:
# Read example data
baudrate = 230400
cport = 'COM3'  # set the correct port before you run it
#cport = '/dev/tty.usbmodem141101'  # set the correct port before run it
ser = serial.Serial(port=cport, baudrate=baudrate)    
# take example data
inputBufferSize = 10000 # 20000 = 1 second
ser.timeout = inputBufferSize/20000.0  # set read timeout

In [6]:
# Visualize data to inspect
data = read_arduino(ser,inputBufferSize)
data_plot = process_data(data)
plt.figure()
plt.plot(data_plot)
plt.show()

In [7]:
# Define stream buffer size
inputBufferSize = 10000 # keep betweein 2000-20000
ser.timeout = inputBufferSize/20000.0  # set read timeout, 20000 is one second
# ser.set_buffer_size(rx_size = inputBufferSize)

# Define steam length
total_time = 5.0; # time in seconds [[1 s = 20000 buffer size]]
max_time = 10.0; # time plotted in window [s]
N_loops = 20000.0/inputBufferSize*total_time

In [13]:
# Run stream

from collections import deque

T_acquire = inputBufferSize/20000.0    # length of time that data is acquired for 
N_max_loops = max_time/T_acquire    # total number of loops to cover desire time window

def running_mean(data, window_size):
    """Compute the running mean over a given window size."""
    cumsum = np.cumsum(np.insert(data, 0, 0))
    return (cumsum[window_size:] - cumsum[:-window_size]) / window_size

# Define the size of the circular buffer
buffer_size = 1000 #4000

# Initialize the circular buffer using a deque
data_buffer = deque(maxlen=buffer_size)

fig, ax = plt.subplots()
ax.set_ylim(-250,250)

count = 0
running_mean_all = []
data_all = []
for k in range(int(N_loops)):
    data = read_arduino(ser, inputBufferSize)
    data_temp = process_data(data)
    data_preproc = np.abs(data_temp - 500)
    
    running_mean_all.append(running_mean(data_preproc, 500))

    # Append the elements of data_temp to the circular buffer
    for item in data_preproc:
        data_buffer.append(item)

    # data_all.append(data_buffer.copy())
    data_all.append(data_preproc.copy())

    if k <= N_max_loops:
        if k==0:
            data_plot = data_temp
        else:
            data_plot = np.append(data_temp,data_plot)
        t = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,len(data_plot))
    else:
        data_plot = np.roll(data_plot,len(data_temp))
        data_plot[0:len(data_temp)] = data_temp
    t = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,len(data_plot))

    if k == 0:
        # line, = ax.plot(t[:-1], np.array(t)[:-1])
        # line, = ax.plot(t, np.array(t))
        line, = ax.plot(np.arange(0,buffer_size), np.arange(0,buffer_size))
    
    
    # ax.plot(t,data_plot)
    line.set_ydata(data_buffer)
    fig.canvas.draw()
    fig.canvas.flush_events()


In [14]:
# Plot running mean activity and STD of running mean

data_h = np.hstack(data_all)
run_mean_h = np.hstack(running_mean_all)

f = interpolate.interp1d(np.arange(len(run_mean_h)),run_mean_h)
x_new = np.linspace(0,len(run_mean_h)-1,len(data_h))
run_mean_interp = f(x_new)

rmSTD = np.mean(run_mean_h)+np.std(run_mean_h)

plt.figure()
plt.plot(data_h,alpha=0.2, label='Raw_data')
plt.plot(run_mean_interp, label='Running_mean')
plt.axhline(rmSTD,c='k',linestyle='--', label='Standard_Dev')
plt.legend()
# plt.show()

print('One standard deviation above the mean is ' + str(rmSTD))
print('Try this number as the threshold in pong_stream.py!')

One standard deviation above the mean is 21.478250502236396
Try this number as the threshold in pong_stream.py!
